In [1]:
import backoff
from typing import Any

BACKOFF_CONFIG: dict[str, Any] = {'wait_gen': backoff.expo, 'exception': Exception, 'max_value': 8}

class ClickhouseSettings:
    NODES: str = 'clickhouse-node01:9000,clickhouse-node02:9000,clickhouse-node03:9000,clickhouse-node04:9000'
    USER: str = 'user'
    PASSWORD: str = 'password'

class ClickhouseNode1:
    HOST: str = 'clickhouse-node01'
    PORT: int = 9000
    USER: str = 'user'
    PASSWORD: str = 'password'

class ClickhouseNode2:
    HOST: str = 'clickhouse-node02'
    PORT: int = 9000
    USER: str = 'user'
    PASSWORD: str = 'password'

class ClickhouseNode3:
    HOST: str = 'clickhouse-node03'
    PORT: int = 9000
    USER: str = 'user'
    PASSWORD: str = 'password'

class ClickhouseNode4:
    HOST: str = 'clickhouse-node04'
    PORT: int = 9000
    USER: str = 'user'
    PASSWORD: str = 'password'

CLICKHOUSE_CONFIG = ClickhouseSettings()
NODES = [ClickhouseNode1(), ClickhouseNode2(), ClickhouseNode3(), ClickhouseNode4()]


In [2]:
clickhouse_url = f'jdbc:clickhouse://{NODES[0].HOST}:{NODES[0].PORT}'
properties = {
    'user': NODES[0].USER,
    'password': NODES[0].PASSWORD,
    'driver': 'com.github.housepower.jdbc.ClickHouseDriver'
}

In [3]:
from pyspark.sql import SparkSession

appName='clickhouse_connector'
spark = SparkSession \
    .builder \
    .appName(appName) \
    .master('spark://spark-master:7077') \
    .config('spark.jars', '/opt/clickhouse-native-jdbc-shaded-2.7.1.jar') \
    .getOrCreate()

23/12/07 20:45:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
query = 'SELECT * from default.candles ORDER BY begin DESC'
d = spark.read.format("jdbc") \
    .option('driver', properties.get('driver')) \
    .option('url', clickhouse_url) \
    .option('user', properties.get('user')) \
    .option('password',  properties.get('password')) \
    .option('query', query) \
    .load()

In [5]:
d.take(15)

[Row(id='da387235-41ee-44da-becb-20b7f48827a7', secid='POSI', open=2006.0, close=2006.0, high=2006.0, low=2006.0, value=42126.0, volume=21.0, begin=datetime.datetime(2023, 12, 7, 16, 20), end=datetime.datetime(2023, 12, 7, 16, 20, 59), created_at=datetime.datetime(2023, 12, 7, 13, 35)),
 Row(id='ad120f17-3a3a-4b06-b282-1c0c04d6fc4b', secid='RTKM', open=71.0, close=70.95, high=71.0, low=70.95, value=100774.4, volume=1420.0, begin=datetime.datetime(2023, 12, 7, 16, 20), end=datetime.datetime(2023, 12, 7, 16, 20, 59), created_at=datetime.datetime(2023, 12, 7, 13, 35)),
 Row(id='30106fef-f459-4b99-b270-94c638981c67', secid='SIBN', open=850.15, close=851.1, high=851.1, low=850.15, value=5274909.349999998, volume=6201.0, begin=datetime.datetime(2023, 12, 7, 16, 20), end=datetime.datetime(2023, 12, 7, 16, 20, 59), created_at=datetime.datetime(2023, 12, 7, 13, 35)),
 Row(id='3a3e5880-de9d-4d29-9fb2-452ba7c859a8', secid='SELG', open=61.16, close=61.06, high=61.3, low=61.06, value=261403.0, volu